In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# The large spreadsheet
data_file = 'WRZ_summary_data.csv'

# Read into pandas data frame ignoring some cols
pd_data = pd.read_csv(
    data_file,
    usecols=lambda col: col.lower().strip() not in [
        'owner'.lower(),
        'wrmp_14_by_19_factor'.lower(),
        'wrmp_14_wrz_name'.lower(),
    ]
)

In [ ]:
pd_data.head()

In [ ]:
legend = pd_data[['ea_code', 'measurement', 'unit']].sort_values(by='ea_code').drop_duplicates()
legend = legend[
    legend.measurement.str.contains('PCC') 
    | legend.measurement.str.contains('Population') 
    | legend.measurement.str.contains('Distribution Input')
    | legend.measurement.str.contains('Distribution input')
]
legend

In [ ]:
def fix_nottinghamshire(record):
    if record['GIS Ref'] == 11:
        return 'Nottinghamshire.1'
    if record['GIS Ref'] == 80:
        return 'Nottinghamshire.2'
    return record['WRZ Name']
pd_data['WRZ Name'] = pd_data.apply(fix_nottinghamshire, axis=1)

In [ ]:
exclude_wrzs = ['Barepot', 'Rutland', 'South Humber Bank', 'Industrial']
data = pd_data[pd_data.ea_code.isin(legend.ea_code) & ~pd_data['WRZ Name'].isin(exclude_wrzs)].copy()
data.value = data.value.astype(float)
data.year = data.year.apply(lambda d: d.split("-")[0]).astype(int)
data = data[['WRZ Name', 'ea_code', 'year', 'value']] \
    .pivot_table(index=['WRZ Name', 'year'], columns='ea_code') \
    .reset_index()
data.columns = [
    col[1] if col[0] == 'value'
    else col[0]
    for col in data.columns
]
data = data.rename(columns={
    'WRZ Name': 'water_resource_zones',
    'year': 'timestep'
})
data

In [ ]:
# Handwritten expected water resource zone names
expected_names = np.asarray([
    'Ashford', 'Berwick', 'Bishops Castle', 'Blyth', 'Bourne', 'Bournemouth',
    'Bracknell', 'Brett', 'Bristol Water', 'Bury Haverhill', 'Cambridge',
    'Carlisle', 'Central Essex', 'Central Lincolnshire', 'Chester',
    'Cheveley', 'Colliford', 'Colne', 'Company', 'Cranbrook', 'Dour',
    'East Lincolnshire', 'East SWZ', 'East Suffolk', 'Eastbourne', 'Ely',
    'Essex', 'Farnham', 'Forest and Stroud', 'Grid SWZ', 'Guildford (GUI)',
    'Hampshire Andover', 'Hampshire Kingsclere', 'Hampshire Rural',
    'Hampshire Southampton East', 'Hampshire Southampton West',
    'Hampshire Winchester', 'Happisburgh', 'Hartismere', 'Hartlepol',
    'Haywards Heath', 'Henley (HEN)', 'Isle of Wight', 'Ixworth',
    'Kennet Valley (KV)', 'Kent Medway East', 'Kent Medway West',
    'Kent Thanet', 'Kielder', 'Kinsall', 'Lee', 'London (LON)', 'Maidstone',
    'Mardy', 'Misbourne', 'Newark', 'Newmarket', 'North Eden',
    'North Fenland', 'North Norfolk Coast', 'North Norfolk Rural',
    'North Staffordshire', 'Northern Central', 'Norwich and the Broads',
    'Nottinghamshire.1', 'Nottinghamshire.2', 'Pinn', 'Roadford',
    'Ruthamford Central', 'Ruthamford North', 'Ruthamford South',
    'Ruthamford West', 'Ruyton', 'SES Water', 'Shelton', 'Slough (SWA)',
    'South Essex', 'South Fenland', 'South Lincolnshire',
    'South Norfolk Rural', 'South Staffs', 'Stafford', 'Stort', 'Strategic',
    'Strategic Grid', 'Sudbury', 'Supply Area', 'Sussex Brighton',
    'Sussex Hastings', 'Sussex North', 'Sussex Worthing', 'Swindon (SWOX)',
    'Thetford', 'Tunbridge Wells', 'Wey', 'Whitchurch and Wem', 'Wimbleball',
    'Wolverhampton'
])

# Check the names match up for a representative slice of the data
names_from_filter = data[data.timestep == 2025]['water_resource_zones'].values
assert np.array_equal(expected_names, names_from_filter), set(names_from_filter) ^ set(expected_names)

In [ ]:
# data[data['WRZ Name'].str.contains('Nottinghamshire')]

In [ ]:
def get_wales():
    # Additional water resource zone names for Wales, not in the spreadsheet
    zones = pd.DataFrame([
        {'water_resource_zones': 'Wye', 'population': 349.3},
        {'water_resource_zones': 'SEWCUS', 'population': 1340.8},
        {'water_resource_zones': 'Tywi CUS', 'population': 752.1},
        {'water_resource_zones': 'Alwen', 'population': 158.7},
        {'water_resource_zones': 'Ross Bulk Supply', 'population': 22.2},
    ])
    # Generate dataframe with constant population over full time range
    dfs = []
    for year in range(2020, 2045):
        df = zones.copy()
        df['timestep'] = year
        dfs.append(df)

    wales = pd.concat(dfs, axis=0)
    return wales

In [ ]:
def calculate(data, scenario):
    df = data.rename(columns={
        '11{}'.format(scenario): 'distribution_input',
        '29{}'.format(scenario): 'pcc_measured', 
        '30{}'.format(scenario): 'pcc_unmeasured', 
        '49{}'.format(scenario): 'pop_measured_non_household',
        '50{}'.format(scenario): 'pop_unmeasured_non_household',
        '51{}'.format(scenario): 'pop_measured_household',
        '52{}'.format(scenario): 'pop_unmeasured_household',
        '53{}'.format(scenario): 'population',
    }).copy()

    # Constant in Ml per day (1e3 scale for pop * 1e-6 scale for Ml = 1e-3)
    df['pop_measured'] = df.pop_measured_non_household + df.pop_measured_household
    df['pop_unmeasured'] = df.pop_unmeasured_non_household + df.pop_unmeasured_household

    df['constant_water_demand'] = df.distribution_input - 1e-3 * (
        df.pcc_measured * df.pop_measured 
        + df.pcc_unmeasured * df.pop_unmeasured
    )

    # Per capita in Ml/(thousand people)/day
    df['per_capita_water_demand'] = (df.distribution_input - df.constant_water_demand) / df.population
    
    # Check that the calculation gives back the original answer
    assert np.all(np.abs(
        df.constant_water_demand 
        + df.per_capita_water_demand * df.population
        - df.distribution_input
    ) < 1e-9)

    # Calculate mean values for extra regions
    mean_pcc = df.per_capita_water_demand.mean()
    mean_constant_demand_per_person = (df.constant_water_demand / df.population).mean()
    
    wales = get_wales() # timestep, water_resource_zones, population
    wales['per_capita_water_demand'] = mean_pcc
    wales['constant_water_demand'] = wales.population * mean_constant_demand_per_person

    england = df[['timestep', 'water_resource_zones', 'population', 'per_capita_water_demand', 'constant_water_demand']]
    
    return pd.concat([england, wales], axis=0, sort=True)

### Output

In [ ]:
for scenario in ('BL', 'FP'):
    scenario_data = calculate(data, scenario)
    
    scenario_data[['timestep', 'water_resource_zones', 'per_capita_water_demand']] \
        .to_csv('per_capita_water_demand__{}.csv'.format(scenario), index=False)

    scenario_data[['timestep', 'water_resource_zones', 'constant_water_demand']] \
        .to_csv('constant_water_demand__{}.csv'.format(scenario), index=False)

    scenario_data[['water_resource_zones']].drop_duplicates() \
        .to_csv('water_resource_zones.csv'.format(scenario), index=False)

## Sense check

Combine scenario data, summarise at national level, compare to reshaped input data.

Note that scenario data as output includes Wales.

In [ ]:
bl = calculate(data, 'BL')
bl['scenario'] = 'BL'
fp = calculate(data, 'FP')
fp['scenario'] = 'FP'
df = pd.concat([bl, fp], axis=0)
df.head()

In [ ]:
total = df.groupby(['scenario', 'timestep']).sum().reset_index()
mean = df.groupby(['scenario', 'timestep']).mean().reset_index()

In [ ]:
before_total = data.groupby('timestep').sum()
before_mean = data.groupby('timestep').mean()

In [ ]:
total.pivot(index='timestep', columns='scenario', values='population').plot()

In [ ]:
before_total[['53BL', '53FP']].plot()

In [ ]:
total.pivot(index='timestep', columns='scenario', values='constant_water_demand').plot()

In [ ]:
mean.pivot(index='timestep', columns='scenario', values='per_capita_water_demand').plot()

In [ ]:
before_mean[['31BL', '31FP']].plot()

In [ ]:
total['mean_pcc'] = mean['per_capita_water_demand']
total['total_demand'] = total.population * total.mean_pcc + total.constant_water_demand

In [ ]:
total.pivot(index='timestep', columns='scenario', values='total_demand').plot()

In [ ]:
before_total[['11BL', '11FP']].plot()